In [49]:
# coding:utf8
#参考：https://www.jianshu.com/p/6ca1344a09db
import re
import requests
import random
from zlib import crc32
from base64 import b64decode
import urllib
import pymongo
import pandas as pd
from os import path
import time

In [50]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTHL, like Gecko) Chrome/66.0.3359.181 Safari/537.36",
}
    
def get_video_url_api(video_id='v02004190000bjqdaaq0ifkkoafl5rg0'):
    '''取视蘋地iff所在包的uri'''
    r = str(random.random())[2:]
    url_part = "/video/urls/v/l/toutiao/mp4/{}?r={}".format(video_id, r) 
    s = crc32(url_part.encode())
    url = "https://ib.365yg.com{}&s={}".format(url_part,s) 
    return url
 
def get_video_url(url):
    #获取视频地址
    resp = requests.get(url, headers=headers) 
    j_resp = resp.json()
    video_url = j_resp['data']['video_list']['video_1']['main_url'] 
    video_url = b64decode(video_url.encode()).decode() 
    return video_url
 
def get_video_id(url):
    #获取视频id
    resp = requests.get(url, headers=headers)
    # 获取video_id
    print(resp.text)
    search = re.search("\"vid\":\"([^\"]+)\",", resp.text)
    print(search.group(1))
    return search.group(1)
 
def main(video_id):
    video_url_api = get_video_url_api(video_id)
    video_url = get_video_url(video_url_api)
    return video_url

def downloadfile(filepath,url):
    r=requests.get(url)
    print("****Connected****")
    f=open(filepath,'wb');
    print("Donloading.....")
    for chunk in r.iter_content(chunk_size=255): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    print("Done")
    f.close()

In [51]:
username_str = 'breadt'
password_str = 'Breadt@2019'

username = urllib.parse.quote_plus(username_str)
password = urllib.parse.quote_plus(password_str)

client = pymongo.MongoClient('mongodb://%s:%s@192.168.31.87:27017/' % (username, password))
db = client["ixigua"]

data = []
for x in db['video_info'].find():
    data.append(x['video_detail_info'])
    
client.close()

In [52]:
df = pd.DataFrame(data)

In [54]:
for index, item in df.iterrows():
    filename = item['video_id'] + '.mp4'
    filepath = '../video/' + filename
    
    if not path.exists(filepath):
        print(item['video_id'])
        url = main(item['video_id'])
        downloadfile(filepath, url)
        time.sleep(10)


v03004770000bjv9teu0gekshtkqocfg
****Connected****
Donloading.....
Done
